# Презентация примеров использования модулей, классов, функций, применяемых в ходе данного курсового проекта.

In [28]:
from model import *
from utils.builder import PandasGraphBuilder
import pandas as pd
import numpy as np

In [29]:
def random_dates(start, end, n=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

## Представление данных в виде графа

### Генерируем данные

In [30]:
articles = pd.DataFrame({'article_id': np.arange(1,101), 'Цвет': np.random.randint(1,10, 100), 'Материал':np.random.randint(1,5, 100)})

start = pd.to_datetime('2015-01-01')
end = pd.to_datetime('2018-01-01')
transactions = pd.DataFrame({
    'customer_id': np.random.randint(1,2000,2000), 
    'article_id': np.random.randint(1, 101, 2000),
    't_dat': pd.date_range(start, end, 2000).values.astype('datetime64[s]').astype('int64')
    })

customers = pd.DataFrame({'customer_id': transactions['customer_id'].drop_duplicates()})

In [31]:
len(customers)

1250

### Создание графа из pd.DataFrame с помощью модуля utils.builder

In [32]:
graph_builder = PandasGraphBuilder()
graph_builder.add_entities(customers, 'customer_id', 'customer')
graph_builder.add_entities(articles, 'article_id', 'article')
graph_builder.add_binary_relations(transactions, 'customer_id', 'article_id', 'bought')
graph_builder.add_binary_relations(transactions, 'article_id', 'customer_id', 'bought-by')
g = graph_builder.build();g

Graph(num_nodes={'article': 100, 'customer': 1250},
      num_edges={('article', 'bought-by', 'customer'): 2000, ('customer', 'bought', 'article'): 2000},
      metagraph=[('article', 'customer', 'bought-by'), ('customer', 'article', 'bought')])

### Добавление признаков вершинам и ребрам

In [33]:
for col in articles.columns:
    if col == 'article_id':
        continue
    else:
        g.nodes['article'].data[col] = torch.LongTensor(articles[col].values)

g.edges['bought'].data['t_dat'] = torch.LongTensor(transactions['t_dat'].values)
g.edges['bought-by'].data['t_dat'] = torch.LongTensor(transactions['t_dat'].values)
g.ndata['Цвет']

{'article': tensor([4, 6, 8, 5, 3, 9, 9, 2, 1, 7, 1, 7, 5, 3, 6, 8, 9, 9, 2, 2, 2, 4, 6, 7,
         9, 3, 2, 3, 9, 7, 9, 9, 2, 3, 7, 7, 5, 1, 5, 9, 9, 1, 4, 2, 9, 7, 2, 6,
         9, 5, 3, 6, 5, 3, 6, 4, 5, 7, 8, 7, 7, 8, 9, 4, 9, 8, 6, 9, 3, 8, 5, 6,
         4, 4, 2, 5, 3, 4, 5, 7, 2, 9, 6, 2, 6, 7, 9, 6, 6, 7, 3, 1, 3, 9, 2, 1,
         9, 7, 5, 6])}

In [34]:
textset = torchtext.legacy.data.Dataset([], {})

## Сэмплирование, создание датасета c помощью модуля utils.sampler

In [35]:
args = {
    'dataset': g,
    'random-walk-length': 2,
    'random-walk-restart-prob': 0.5,
    'num-random-walks': 10, 
    'num-neighbors': 3,
    'num-layers': 2,
    'hidden-dims': 64,
    'batch-size': 16,
    'device': 'cpu',
    'num-epochs': 5,
    'batches-per-epoch': 200,
    'num-workers': 0,
    'lr': 1e-6,
    'k': 3
}
device = args['device']

### Генератор Батчей. Батчм включают в себя heads - N случайных узлов, tails - N cоседей heads, полученных с помощью случайного блуждания, neg_tails - негативные узлы (не соседи), отобранные случайно, т.к для больших графов вероятность того, что случайно отобранная вершина окажется негативным примером, приблизительно равна 100%. На полученных данных будет считаться функция потерь.

In [36]:
batch_sampler = sampler_module.ItemToItemBatchSampler(
        g, 'customer', 'article', args['batch-size'])
next(batch_sampler.__iter__())

(tensor([93, 56, 22, 13, 94, 85,  9, 55, 50,  0,  8, 55, 66, 57, 84, 59]),
 tensor([93, 56, 22, 13, 94, 36,  9, 55, 50,  0,  8, 55, 86, 57, 18, 59]),
 tensor([48, 32, 45, 88, 86, 91,  6,  4, 50,  7, 70, 29, 10, 72, 88, 22]))

### Sampler создает pos_graph и neg_graph - графы, узлы которого являются heads, tails, neg_tails. В pos_graph узлы, которые являются соседями, соединены ребрами, в neg_graph наоборот. Так же sampler создает dgl.Block - специальная структура данных/граф, созданная для передачи сообщений между узлами. 

In [37]:
neighbor_sampler = sampler_module.NeighborSampler(
    g, 'customer', 'article', args['random-walk-length'],
    args['random-walk-restart-prob'], args['num-random-walks'], args['num-neighbors'],
    args['num-layers'])
collator = sampler_module.PinSAGECollator(neighbor_sampler, g, 'article', textset)

In [38]:
dataloader = DataLoader(
        batch_sampler,
        collate_fn=collator.collate_train,
        num_workers=args['num-workers'])

dataloader_test = DataLoader(
        torch.arange(g.number_of_nodes('article')),
        batch_size=args['batch-size'],
        collate_fn=collator.collate_test,
        num_workers=args['num-workers'])

In [39]:
"""
batch = [next(batch_sampler.__iter__())]
collator.collate_train(batch)
"""

dataloader_it = iter(dataloader)
pos_graph, neg_graph, blocks = next(dataloader_it)

In [40]:
pos_graph

Graph(num_nodes=31, num_edges=16,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [41]:
neg_graph

Graph(num_nodes=31, num_edges=16,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [42]:
blocks

[Block(num_src_nodes=85, num_dst_nodes=60, num_edges=170),
 Block(num_src_nodes=60, num_dst_nodes=31, num_edges=82)]

## Обучение модели, получение новых представлений вершин

### Определение модели и оптимизатора

In [43]:
model = PinSAGEModel(g, 'article', textset, args['hidden-dims'], args['num-layers']).to(device)
opt = torch.optim.SGD(model.parameters(), lr=args['lr'])

### Обучение модели

In [44]:
for epoch_id in range(args['num-epochs']):
    loss_batch = []
    model.train()
    for batch_id in tqdm.trange(args['batches-per-epoch']):
        pos_graph, neg_graph, blocks = next(dataloader_it)            
        for i in range(len(blocks)):
            blocks[i] = blocks[i].to(device)
        pos_graph = pos_graph.to(device)
        neg_graph = neg_graph.to(device)
        loss = model(pos_graph, neg_graph, blocks).mean()
        loss_batch.append(loss.item())
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f'Epo {epoch_id + 1}: {np.mean(loss_batch)}')

  0%|          | 0/200 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|██████████| 200/200 [00:04<00:00, 43.58it/s]


Epo 1: 0.43159329069778324


100%|██████████| 200/200 [00:04<00:00, 47.19it/s]


Epo 2: 0.4660799918882549


100%|██████████| 200/200 [00:03<00:00, 64.29it/s]


Epo 3: 0.4554502713307738


100%|██████████| 200/200 [00:03<00:00, 63.05it/s]


Epo 4: 0.43956402331590655


100%|██████████| 200/200 [00:03<00:00, 65.42it/s]

Epo 5: 0.46868302853778004


### Получение новых представлений вершин

In [45]:
with torch.no_grad():
        item_batches = torch.arange(g.number_of_nodes('article')).split(args['batch-size'])
        h_item_batches = []
        for blocks in dataloader_test:
            for i in range(len(blocks)):
                blocks[i] = blocks[i].to(device)

            h_item_batches.append(model.get_repr(blocks))
        h_item = torch.cat(h_item_batches, 0)

### Рекомендации

In [46]:
def get_recommendations(h_item, K):
    recommendations = []
    for i in range(len(h_item)):
        dist = h_item[i] @ h_item.T
        dist[i] = -np.inf
        recommendations.append(torch.topk(dist, K)[1])
    return torch.stack(recommendations)

In [47]:
recomendations = get_recommendations(h_item, 3); recomendations[:5]

tensor([[72, 77, 42],
        [82, 36, 49],
        [58, 61, 69],
        [70, 98, 78],
        [25, 33, 53]])

In [48]:
neighbor_sampler.sample_blocks([0,1,2,3,4])[-1].all_edges()

(tensor([ 0,  5,  6,  1,  7,  8,  2,  9, 10,  3, 11, 12,  4, 13, 14]),
 tensor([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]))

In [49]:
articles.iloc[0]

article_id    1
Цвет          4
Материал      1
Name: 0, dtype: int64

In [52]:
articles.iloc[72]

article_id    73
Цвет           4
Материал       1
Name: 72, dtype: int64